In [ ]:
!pip install pillow

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
pip install pyheif

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 14.2 MB/s eta 0:00:00


In [ ]:
import os
import pyheif
from PIL import Image

Converting .HEIF files to .jpg

In [ ]:


def convert_heic_to_jpg(heic_path, jpg_path):
    heif_file = pyheif.read(heic_path)
    image = Image.frombytes(
        heif_file.mode,
        heif_file.size,
        heif_file.data,
        "raw",
        heif_file.mode,
        heif_file.stride,
    )
    image.save(jpg_path, "JPEG")

In [ ]:
def convert_all_heic_to_jpg(folder_path):
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(".heic"):
            heic_path = os.path.join(folder_path, filename)
            jpg_path = os.path.join(folder_path, os.path.splitext(filename)[0] + ".jpg")

            convert_heic_to_jpg(heic_path, jpg_path)

if __name__ == "__main__":
    folder_path = "/content/drive/MyDrive/MLPR Project/New_Data/new_data"
    convert_all_heic_to_jpg(folder_path)

Cropping Annotated Images

In [ ]:
import os
from xml.etree import ElementTree as ET
from PIL import Image
from collections import defaultdict

def crop_and_save_images(xml_folder, raw_data_folder, output_folder):
    for xml_file in os.listdir(xml_folder):
        if xml_file.endswith('.xml'):
            xml_path = os.path.join(xml_folder, xml_file)

            try:
                tree = ET.parse(xml_path)
                root = tree.getroot()

                image_filename = root.find('filename').text
                image_path = os.path.join(raw_data_folder, image_filename)

                print(f"Processing {xml_file}: Found image path: {image_path}")

                if not os.path.exists(image_path):
                    print(f"Error: Image file not found for {xml_file}")
                    continue

                image = Image.open(image_path)

                label_count = defaultdict(int)

                for obj in root.findall('.//object'):
                    label = obj.find('name').text
                    bndbox = obj.find('bndbox')

                    xmin = int(bndbox.find('xmin').text)
                    ymin = int(bndbox.find('ymin').text)
                    xmax = int(bndbox.find('xmax').text)
                    ymax = int(bndbox.find('ymax').text)

                    cropped_image = image.crop((xmin, ymin, xmax, ymax))

                    label_folder = os.path.join(output_folder, label)
                    os.makedirs(label_folder, exist_ok=True)

                    # Increment the count for this label
                    label_count[label] += 1

                    # Modify the file name to include the count
                    cropped_image_filename = f"{image_filename.split('.')[0]}_{label}_{label_count[label]}.png"
                    cropped_image.save(os.path.join(label_folder, cropped_image_filename))

            except Exception as e:
                print(f"Error processing {xml_file}: {e}")

if __name__ == "__main__":
    xml_folder = "/content/drive/MyDrive/MLPR Project/Data/labels_mlpr"
    raw_data_folder = "/content/drive/MyDrive/MLPR Project/Data/raw_data"
    output_folder = "/content/drive/MyDrive/MLPR Project/Data/final_cropped"

    crop_and_save_images(xml_folder, raw_data_folder, output_folder)

Processing 14.xml: Found image path: /content/drive/MyDrive/MLPR Project/Data/raw_data/14.jpg
Processing 18.xml: Found image path: /content/drive/MyDrive/MLPR Project/Data/raw_data/18.jpg
Processing 25.xml: Found image path: /content/drive/MyDrive/MLPR Project/Data/raw_data/25.jpg
Processing 23.xml: Found image path: /content/drive/MyDrive/MLPR Project/Data/raw_data/23.jpg
Processing 29.xml: Found image path: /content/drive/MyDrive/MLPR Project/Data/raw_data/29.jpg
Processing 30.xml: Found image path: /content/drive/MyDrive/MLPR Project/Data/raw_data/30.jpg
Processing 24.xml: Found image path: /content/drive/MyDrive/MLPR Project/Data/raw_data/24.jpg
Processing 13.xml: Found image path: /content/drive/MyDrive/MLPR Project/Data/raw_data/13.jpg
Processing 16.xml: Found image path: /content/drive/MyDrive/MLPR Project/Data/raw_data/16.jpg
Processing 3.xml: Found image path: /content/drive/MyDrive/MLPR Project/Data/raw_data/3.jpg
Processing 1.xml: Found image path: /content/drive/MyDrive/MLP